
# Vertex AI – Gemini Quickstart (Colab/Notebook)

This notebook shows how to use **`google-genai`** with an **API key** to call Gemini on Vertex AI.
- Works in **Colab** or locally (Jupyter).
- Includes **streaming** and **non-streaming** calls, JSON-structured output, and quick helpers.

> **Prereqs**  
> 1) In project `starry-argon-463819-a2`, create an **API key** with GenAI access.  
> 2) Paste it into the cell below.  
> 3) Ensure the model ID is available in your project.


In [ ]:

# Install the SDK
!pip -q install --upgrade google-genai


In [ ]:

# 🔑 Set your API key here (temporary for this runtime only)
import os

GOOGLE_CLOUD_API_KEY = "PASTE_YOUR_KEY_HERE"  # <-- replace
os.environ["GOOGLE_CLOUD_API_KEY"] = GOOGLE_CLOUD_API_KEY

PROJECT_ID = "starry-argon-463819-a2"
REGION = "us-central1"

assert os.environ.get("GOOGLE_CLOUD_API_KEY") and os.environ["GOOGLE_CLOUD_API_KEY"] != "PASTE_YOUR_KEY_HERE",     "Please paste a real API key above."
print("API key is set in the environment.")


In [ ]:

# 🧠 Client bootstrap
from google import genai
from google.genai import types
import os

MODEL_ID = "gemini-2.5-flash-preview-09-2025"  # update if needed

client = genai.Client(
    vertexai=True,  # use Vertex AI endpoint
    api_key=os.environ["GOOGLE_CLOUD_API_KEY"],
)

print("Client ready. Model:", MODEL_ID)



## 1) Non-streaming: basic text generation


In [ ]:

prompt = "List three whimsical use cases for a white-label AI content engine."
cfg = types.GenerateContentConfig(
    temperature=0.9,
    top_p=0.95,
    max_output_tokens=512,
    system_instruction=[types.Part.from_text(text=(
        "You are a helpful creative system that produces useful, remixable assets quickly."
    ))],
)

resp = client.models.generate_content(
    model=MODEL_ID,
    contents=[types.Content(role="user", parts=[types.Part.from_text(text=prompt)])],
    config=cfg,
)

print(resp.text)



## 2) Streaming: token-by-token output


In [ ]:

cfg_stream = types.GenerateContentConfig(
    temperature=1.0,
    top_p=0.95,
    max_output_tokens=1024,
)

print("Streaming response...")
for chunk in client.models.generate_content_stream(
    model=MODEL_ID,
    contents=[types.Content(role="user", parts=[types.Part.from_text(text="Write a one-paragraph product pitch for a modular AI demo kit.")])],
    config=cfg_stream,
):
    if chunk.text:
        print(chunk.text, end="")
print("

— end of stream —")



## 3) JSON-structured output
Ask the model to conform to a simple JSON schema for downstream use.


In [ ]:

schema = {
  "type": "object",
  "properties": {
    "title": {"type": "string"},
    "bullets": {"type": "array", "items": {"type": "string"}},
    "cta": {"type": "string"}
  },
  "required": ["title", "bullets", "cta"]
}

cfg_json = types.GenerateContentConfig(
    temperature=0.8,
    response_mime_type="application/json",
    response_schema=schema
)

resp_json = client.models.generate_content(
    model=MODEL_ID,
    contents=[types.Content(role="user", parts=[types.Part.from_text(text="Create a landing blurb for an AI-powered slide deck generator.")])],
    config=cfg_json,
)

print(resp_json.text)  # should be valid JSON



## 4) Safety + long outputs (use carefully)
Preview models allow configuring safety thresholds. Here we disable most checks for fast prototyping (adjust for prod).


In [ ]:

cfg_loose = types.GenerateContentConfig(
    temperature=1.0,
    top_p=0.95,
    max_output_tokens=4096,
    safety_settings=[
        types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
    ],
)

resp_long = client.models.generate_content(
    model=MODEL_ID,
    contents=[types.Content(role="user", parts=[types.Part.from_text(text="Draft a 6-section outline for a product launch brief aimed at enterprise buyers.")])],
    config=cfg_loose,
)

print(resp_long.text[:2000])  # preview first ~2k chars



## 5) Pro tip: switch models quickly
Just change `MODEL_ID` (ensure it's enabled for your project). Common choices include `gemini-2.0-flash`, `gemini-2.0-pro`, or later previews.
